In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
print(os.listdir("../input/santander-test1"))


In [ ]:
import tensorflow as tf
device_name = tf. test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
%%time
base_dir = '../input/santander-customer-transaction-prediction/'
base_train_df = pd.read_csv(base_dir + "train.csv")
test_df = pd.read_csv(base_dir + "test.csv")
smple_df = pd.read_csv(base_dir + "sample_submission.csv")

In [ ]:
test2_df = pd.read_csv("../input/santander-test1/" + "submission2.csv")
test2_df_all = pd.merge(test2_df,test_df,on=['ID_code'])
#all_df=pd.concat([base_train_df,test2_df_all])
#all_df.shape

In [ ]:
print('train_df.shape = {}'.format(base_train_df.shape))
print('test_df.shape = {}'.format(test_df.shape))

In [ ]:
#Сформируем отложенную выборку
base_train_df=base_train_df.sample(frac=1)
train_df_train=base_train_df.iloc[:180000]
train_df_test=base_train_df.iloc[180000:]
print(train_df_train.shape)
print(train_df_test.shape)

In [ ]:
#дополнение выборки до сбалансированной, 
if 1:
    train_df_target = train_df_train
    for i in range(7):
        train_df_target=pd.concat([train_df_target,train_df_train[train_df_train['target']==1]])
    train_df_target = train_df_target.sample(frac=1)
    print(train_df_target.shape)
    
# или использование читерской выборки, без отложенной части
if 0:
    train_df_target = pd.concat([train_df_train,test2_df_all])
    train_df_target = train_df_target.sample(frac=1)
    print(train_df_target.shape)

# или стандартный тренировочный набор
if 0:
    train_df_target = train_df_train
    print(train_df_target.shape)

In [ ]:
n1=train_df_target[train_df_target['target']==1].shape[0]
n0=train_df_target[train_df_target['target']==0].shape[0]
n=train_df_target.shape[0]
print(train_df_target[train_df_target['target']==1].shape)
print(train_df_target[train_df_target['target']==0].shape)
print(n1/n,n0/n)
train_df_target.describe()

In [ ]:
x_test = test_df[test_df.columns[1:]].values
id_test = test_df[test_df.columns[0:]].values


In [ ]:
x_train = train_df_target[train_df_target.columns[2:]].values
y_train = train_df_target[train_df_target.columns[1]].values
x_val = train_df_test[train_df_test.columns[2:]].values
y_val = train_df_test[train_df_test.columns[1]].values

mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std

x_val -= mean
x_val /= std

x_test = test_df[test_df.columns[1:]].values
id_test = test_df[test_df.columns[0]].values
x_test -= mean
x_test /= std

print(x_train.shape)
print(x_val.shape)

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras import metrics
from sklearn.metrics import auc
from keras import regularizers

model = models.Sequential()
n = 64
model.add(layers.Dense(n, 
                       activation='relu', 
                       kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),
                       #kernel_regularizer=regularizers.l1(0.001),
                       #bias_regularizer=regularizers.l2(0.001),
                       #activity_regularizer=regularizers.l2(0.001),
                       
                       input_shape=(200,)))
model.add(layers.Dense(n, 
                       activation='relu', 
                       kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),
                       #kernel_regularizer=regularizers.l1(0.001),
                       #bias_regularizer=regularizers.l2(0.001),
                       #activity_regularizer=regularizers.l2(0.001),

                       ))
model.add(layers.Dense(n, 
                       activation='relu', 
                       kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),
                       #kernel_regularizer=regularizers.l1(0.001),
                       #bias_regularizer=regularizers.l2(0.001),
                       #activity_regularizer=regularizers.l2(0.001),

                       ))

model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
                loss='binary_crossentropy',
                metrics=['accuracy'])

model.summary()



In [ ]:
history = model.fit(x_train, 
                    y_train, 
                    epochs=50, # 20 итераций по всем образцам partial_x_train
                    batch_size=512, # пакетами по 512 образцов
                    #validation_data=(x_val, y_val),
                    validation_split=0.2,
                    verbose=1)

In [ ]:
#x_train, y_train, x_val, y_val
y_train_pred = model.predict(x_train)
y_val_pred = model.predict(x_val)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score, auc
print('base: roc_auc_score = ',roc_auc_score(y_train, np.zeros(len(y_train))))
print('train: roc_auc_score = ',roc_auc_score(y_train, y_train_pred))
print('test: roc_auc_score = ',roc_auc_score(y_val, y_val_pred))



In [ ]:
# Листинг 5.10. Формирование графиков изменения потерь и точности в процессе обучения
import matplotlib.pyplot as plt

e1=0
acc = history.history['acc'][e1:] 
val_acc = history.history['val_acc'][e1:] 
loss = history.history['loss'][e1:] 
val_loss = history.history['val_loss'][e1:] 

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras import metrics
from sklearn.metrics import auc
from keras import regularizers

x_train_re=x_train.reshape(130000,200,1)

model = models.Sequential()
model.add(layers.Conv1D(1, 3, input_shape=(200,1), activation='relu'))
#model.add(layers.MaxPooling1D(3))
#model.add(layers.Conv1D(1, 3, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
                loss='binary_crossentropy',
                metrics=['accuracy'])
model.summary()

In [ ]:
x_train_re.shape

In [ ]:
history = model.fit(x_train_re, 
                    y_train, 
                    epochs=30, # 20 итераций по всем образцам partial_x_train
                    batch_size=128, # пакетами по 512 образцов
                    #validation_data=(x_val, y_val),
                    validation_split=0.2,
                    verbose=1)

In [ ]:
predictions = model.predict(x_test)

In [ ]:
sub_df = pd.DataFrame({"ID_code":id_test})
sub_df["target"] = predictions
sub_df.to_csv("submission.csv", index=False)
from IPython.display import FileLink
FileLink('submission.csv')

In [ ]:
print(os.listdir("../input"))